Here, we use the `elf.segmentation` module for boundary and prior based lifted multicut segmentation, as described in [Leveraging Domain Knowledge to Improve Microscopy Image Segmentation With Lifted Multicuts](https://doi.org/10.3389/fcomp.2019.00006).
You can obtain the data from [here](https://oc.embl.de/index.php/s/kzdYaPlmr2NWCni).

The segmentation approach works as follows:
1. Predict pixel-wise affinity or boundary maps. Here, we use pre-computed results for this step.
2. Compute a watershed oversegmentation based on the boundary maps.
3. Compute the region adjacency graph defined by the watershed over-segmentation.
4. Compute weights for the edges of this graph by accumulating the affinity (or boundary) map over the edge pixels.
5. a
6. Partition the graph based on the edge weights via Multicut and project the result back to the pixel level.

In [ ]:
%gui qt5 
import numpy as np

# import napari for data visualisation
import napari

# import the segmentation functionality from elf
import elf.segmentation.multicut as mc
import elf.segmentation.lifted_multicut as lmc
import elf.segmentation.features as feats
import elf.segmentation.watershed as ws

# import the open_file function from elf, which supports opening files
# in hdf5, zarr, n5 or knossos file format
from elf.io import open_file

In [ ]:
# read the data
# you can download the example data from here:
# https://oc.embl.de/index.php/s/kzdYaPlmr2NWCni
data_path = '/home/pape/Work/data/mmwc/knott_data.h5'  # adjust this path
with open_file(data_path, 'r') as f:
    # load the raw data
    raw = f['raw'][:]
    # load the membrane probability maps
    pmap = f['probs/'][:]
    # load the dendrite and vesicle probability maps
    den_map = f['probs/'][:]
    ves_map = f['probs/'][:]

In [ ]:
# visualize the input data with napari
napari.view_image(raw, name='raw')
napari.view_image(pmap, name='membrane-probabilities')
napari.view_image(den_map, name='dendrite-probabilities')
napari.view_image(ves_map, name='vesicle-probabilities')

In [ ]:
# compute the watershed
watershed = ws.distance_transform_watershed(pmap, threshold=.5, 
                                            sigma_seeds=2.)

In [ ]:
# inspect the watershed result
napari.view_image(raw, name='raw')
napari.add_labels(watershed, name='watershed')

In [ ]:
# compute the region adjacency graph
rag = feats.compute_rag(watershed)

# compute the edge costs
features = feats.compute_boundary_mean_and_length(rag, pmap)
costs, sizes = features[:, 0], features[:, 1]

# transform the edge costs from [0, 1] to  [-inf, inf], which is
# necessary for the multicut. This is done by intepreting the values
# as probabilities for an edge being 'true' and then taking the negative log-likelihood.
# in addition, we weight the costs by the size of the corresponding edge
costs = mc.transform_probabilities_to_costs(costs, edge_sizes=sizes)

In [ ]:
# solve the multicut for a baseline
node_labels = mc.multicut_kernighan_lin(rag, costs)
# map the results back to pixels to obtain the final segmentation
segmentation = feats.project_node_labels_to_pixels(rag, node_labels)

In [ ]:
# compute lifted multicut features from vesicle and dendrite pmaps
input_maps = [den_map, ves_map]
assignment_threshold = .8
lifted_uvs, lifted_costs = feats.lifted_problem_from_probabilities(rag, watershed,
                                                                   input_maps, assginment_threshold)
node_labels = lmc.lifted_multicut_kernighan_lin(rag, costs,
                                                lifted_uvs, lifted_costs)
lifted_segmentation = feats.project_node_labels_to_pixes(rag, node_labels)

In [ ]:
# compare segmentations
napari.view_image(raw, name='raw')
napari.add_labels(segmentation, name='multicut-segmentation')
napari.add_labels(lifted_segmentation, name='lifted-multicut-segmentation')